# LMs can handle linear combinations of prompts
We survey a range of transformers, including:
- Eleuther models
- OPT models
- SOLU models
- GPT-2, both small and XL
- Vicuna, a 13B finetuned model **???**

In [ ]:
# Imports
try:
    import algebraic_value_editing
except ImportError:
    commit = "15bcf55"  # Stable commit
    get_ipython().run_line_magic(  # type: ignore
        magic_name="pip",
        line=(
            "install -U"
            f" git+https://github.com/montemac/algebraic_value_editing.git@{commit}"
        ),
    )

In [ ]:
import torch
import pandas as pd
from typing import List, Dict

from transformer_lens.HookedTransformer import HookedTransformer

from algebraic_value_editing import hook_utils, prompt_utils
from algebraic_value_editing.prompt_utils import ActivationAddition

# %%
device: str = "cpu"
model_name = "gpt2-xl"
model: HookedTransformer = HookedTransformer.from_pretrained(
    model_name, device="cpu"
)
_ = model.to(device)

_ = torch.set_grad_enabled(False)
torch.manual_seed(0)  # For reproducibility

## Starting off with GPT-2 small
We use "activation additions" to combine prompts.